In [1]:
!pip3 install   bs4 lxml numpy pandas matplotlib pygsheets \
                gspread requests_html urllib3 selenium \
                msedge-selenium-tools webdriver_manager openpyxl


[notice] A new release of pip is available: 23.2.1 -> 24.0
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from bs4 import BeautifulSoup as bs #to scrapping website
import requests #to make a request acces for scrapping 
import numpy as np #to make an array/slice
import pandas as pd #to manipulate data
import gspread #to push to gspreadsheet
import pygsheets #API to gspreadsheet
import re
import urllib
from tqdm import tqdm
import os
import urllib.request
from urllib.request import urlopen
import selenium
from selenium import webdriver
from selenium.webdriver.common.by import By
from msedge.selenium_tools import Edge, EdgeOptions
from webdriver_manager.microsoft import EdgeChromiumDriverManager
from selenium.webdriver.chrome.options import Options
import time
import openpyxl

In [96]:
# Configure Edge options
options = EdgeOptions()
options.use_chromium = True  # Use Chromium-based Edge
options.add_argument('--headless')
options.add_argument('--disable-gpu')

In [97]:
# Initialize Edge WebDriver using the EdgeChromiumDriverManager and options
driver = Edge(EdgeChromiumDriverManager().install(), options=options)

C:\Users\bobse\AppData\Local\Temp\ipykernel_41896\3888786914.py:2: DeprecationWarning: Selenium Tools for Microsoft Edge is deprecated. Please upgrade to Selenium 4 which has built-in support for Microsoft Edge (Chromium): https://docs.microsoft.com/en-us/microsoft-edge/webdriver-chromium/#upgrading-from-selenium-3
  driver = Edge(EdgeChromiumDriverManager().install(), options=options)


In [98]:
url = "https://www.huntstreet.com/shop/all/{}?type={}&page={}"

In [110]:
Photos = []
Category = []
Sub_Category = []
Current_Prices = []

category = ['bags', 'shoes', 'accessories', 'clothing', 'beauty']

for categories in category:
    for sub_categories in tqdm(range(1,14)):
        for pages in tqdm(range(1,140)):
            driver.get(url.format(categories, sub_categories, pages))
            # time.sleep(2)
            try:
                for index in range(1,40):
                    photo = driver.find_element(by=By.XPATH, value=f'//*[@id="v-product-list"]/div[2]/div[4]/div[{index}]/div[1]/div/a/img').get_attribute('src')
                    try:
                        current_price = driver.find_element(by=By.XPATH, value=f'//*[@id="v-product-list"]/div[2]/div[4]/div[{index}]/div[2]/div[3]/span[2]').text
                    except:
                        current_price = driver.find_element(by=By.XPATH, value=f'//*[@id="v-product-list"]/div[2]/div[4]/div[{index}]/div[2]/div[3]/span').text
                    Photos.append(photo)
                    Category.append(categories)
                    Sub_Category.append(sub_categories)
                    Current_Prices.append(current_price)
            except:
                continue

100%|██████████| 13/13 [38:34<00:00, 178.01s/it]


In [111]:
# url_label = "https://www.huntstreet.com/shop/all/{}"

In [112]:
# Label = []
# Name = []

# category = ['bags', 'shoes', 'accessories', 'clothing', 'beauty']
# category_index = {'bags': 1, 'shoes': 2, 'accessories': 3, 'clothing': 4, 'beauty': 5}

# for categories in category:
#     driver.get(url_label.format(categories))
#     time.sleep(10)
#     try:
#         index = category_index[categories]
#         xpath_index = f'//*[@id="v-product-list"]/div[1]/div[1]/div[3]/div[2]/div[{index}'
#         for inner in range(1, 60):
#             xpath_inner = xpath_index + f']/div[2]/ul/li[{inner}]/label'

#             label = driver.find_element(by=By.XPATH, value=xpath_inner).get_attribute("for")
#             name = driver.find_element(by=By.XPATH, value=xpath_inner).get_attribute("outerHTML")
#             Label.append(label)
#             Name.append(name)
#     except:
#         continue

# Label = [int(label.split('-')[1]) for label in Label]
# Name = [name.split('>')[1].split('<')[0] for name in Name]

In [113]:
hunstreet_data = pd.DataFrame({
    'Category' : Category,
    'Sub_Category' : Sub_Category,
    'Current_Price' : Current_Prices,
    'Photo' : Photos
})

# label_data = dict(zip(Label, Name))

In [114]:
# label_data

{2: 'Tote',
 3: 'Briefcase',
 4: 'Crossbody',
 5: 'Backpack',
 6: 'Luggage &amp; Travel',
 7: 'Clutch',
 1: 'Shoulder',
 12: 'Pumps',
 8: 'Boots',
 9: 'Flats',
 10: 'Loafers',
 11: 'Oxfords',
 13: 'Sandals',
 14: 'Wedges',
 15: 'Sneakers',
 23: 'Watches',
 24: 'Keychains &amp; Charms',
 25: 'Others',
 22: 'Wallets &amp; Card Holders',
 36: 'Baby Gear',
 37: 'Pouch',
 38: 'Beauty',
 16: 'Belts',
 17: 'Jewelry',
 18: 'Headgear',
 19: 'Scarves',
 20: 'Eyewear',
 21: 'Ties',
 33: 'Suit',
 26: 'Dresses',
 27: 'Tops',
 28: 'Outerwear',
 29: 'Pants',
 30: 'Jeans',
 31: 'Shorts',
 32: 'Skirt',
 34: 'Onesie',
 35: 'Set',
 39: 'Make Up',
 40: 'Perfume',
 41: 'Candles',
 42: 'Skincare'}

In [116]:
# Map Sub_Category to Name using label_data
# hunstreet_data['New_SubCategory'] = hunstreet_data['Sub_Category'].map(label_data)

In [117]:
hunstreet_data['Current_Price'] = hunstreet_data['Current_Price'].astype(str)
hunstreet_data['Current_Price'] = hunstreet_data['Current_Price'].str.replace(',','',regex=True)
hunstreet_data['Current_Price'] = hunstreet_data['Current_Price'].str.replace('IDR','',regex=True).str.strip()

In [119]:
# Create a directory to store the images
os.makedirs('images_subcategory', exist_ok=True)

# Download and rename each image
for index, row in hunstreet_data.iterrows():
    file_name = row['New_SubCategory']
    image_url = row['Photo']
    response = requests.get(image_url)
    if response.status_code == 200:
        with open(f'images_subcategory/{file_name}_{index}.jpg', 'wb') as file:
            file.write(response.content)
    else:
        print(f"Failed to download image for '{file_name}'")

print("Images downloaded and renamed successfully.")

Images downloaded and renamed successfully.
